In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb

In [ ]:
%cd drive
%cd 'My Drive'
%cd 'RL'
%cd 'Homework'
%cd 'Homework 4'


In [6]:
'''Needed Libraries and Environment'''
import Environment
import Modified_Environment
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from collections import defaultdict
import sys
import itertools
import matplotlib
import pandas as pd
import pickle

'''Constants Defined'''
NEXTSTATE = 0
STATE = 1
ACTION = 2
REWARD = 3
MAX_T = 1000
TOTAL_EPISODES = 2000
gamma = 0.9
DECAY_FACTOR = 0.99
MIN = 0.001
ALPHA = 0.2

In [7]:
class DoubleQlearningAgent():
  
    def __init__(self, env, gamma):
      self.env = env
      self.Q = defaultdict(lambda: np.zeros(env.action_space.n))
      self.Q_A = defaultdict(lambda: np.zeros(env.action_space.n))
      self.Q_B = defaultdict(lambda: np.zeros(env.action_space.n))
      self.actions = env.action_space.n
      self.gamma = gamma
      self.DQ_episode_scores = []


    def epsilon_greedy_policy(self, state, epsilon):
        epsilon_policy = np.ones(self.actions, dtype=float) * epsilon / self.actions
        best_action = np.argmax(self.Q_A[state] + self.Q_B[state])
        epsilon_policy[best_action] += (1.0 - epsilon)
        return epsilon_policy


    def reach_island(self):
        #epsilon = 1 #<-- uncomment if you want to use decaying epsilon 
        #epsilon = 0.3 #<-- uncomment if you want to use constant epsilon
        for episode in range(1, TOTAL_EPISODES + 1):
            episode_score = 0
            if episode % 2 == 0:
                print("\rEpisode {}/{}.".format(episode, TOTAL_EPISODES), end="")
                #epsilon = max(epsilon * DECAY_FACTOR, MIN)#<-- uncomment if you want to use decaying epsilon
                sys.stdout.flush()
            state = self.env.reset()
            done = False
            for t in itertools.count():
                probs = self.epsilon_greedy_policy(state, epsilon)
                action = np.random.choice(np.arange(len(probs)), p = probs)
                next_state, reward, done, _ = self.env.step(action)
                episode_score += reward
                chosen_number = random.randint(1,2)
                information = [next_state, state, action, reward]
                self.update_chosen_Q(information, chosen_number)
                if done or t > MAX_T :break
                self.Q[state][action] = (self.Q_A[state][action] + self.Q_B[state][action])
                state = next_state
                sys.stdout.flush()
            
            self.DQ_episode_scores.append(episode_score)

    def update_chosen_Q(self, info, chosen_number):
        if chosen_number == 1 :
            self.Q_A[info[STATE]][info[ACTION]] += ALPHA * (info[REWARD] + self.gamma * self.Q_B[info[NEXTSTATE]][np.argmax(self.Q_A[info[NEXTSTATE]][:])] - self.Q_A[info[STATE]][info[ACTION]])
        elif chosen_number == 2:
            self.Q_B[info[STATE]][info[ACTION]] += ALPHA * (info[REWARD] + self.gamma * self.Q_A[info[NEXTSTATE]][np.argmax(self.Q_B[info[NEXTSTATE]][:])] - self.Q_B[info[STATE]][info[ACTION]])



In [8]:
# env =  Environment.GridworldEnv() #<--uncomment for normal environment
# env =  Modified_Environment.GridworldEnv() # <--uncomment for modified environment (bonus part)
agent = DoubleQlearningAgent(env, gamma)
agent.reach_island()

Episode 2000/2000.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(TOTAL_EPISODES), agent.DQ_episode_scores, color = '#633974', label = '$\epsilon$ = from $1$ to $0.001$')
plt.xlabel('episodes ->')
plt.ylabel('epsiode score ->')
plt.title('Double Q-learning')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
window_size = 50
smoothed_score = pd.Series(agent.DQ_episode_scores).rolling(window_size , min_periods=window_size).mean()
plt.plot(smoothed_score, color = '#633974', label = '$\epsilon$ = from $1$ to $0.001$')
#plt.annotate(smoothed_score.iloc[-1], xy=(2000, smoothed_score.iloc[-1]), xytext=(2000, smoothed_score.iloc[-1]), color = '#633974',) # <--uncomment for modified environment (bonus part)
plt.xlabel("epsiode ->")
plt.ylabel("epsiode score (smoothed) -> ")
plt.title('Double Q-learning')
plt.legend()
plt.show()

In [11]:
#i wrote my reward data into a .txt file to use it later.

# with open('DQ_bonus_scores', 'wb') as fp:
#     pickle.dump(agent.DQ_episode_scores, fp)